In [ ]:
!pip install -r ../requirements.in

In [ ]:
!pip install ragas

First, we'll try using langchain and OpenAI so we can learn

In [7]:
import os
from dotenv import load_dotenv
from loguru import logger
from pathlib import Path

load_dotenv(dotenv_path=".env", verbose=True, override=True)

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [10]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("../embed_docs/")
documents = loader.load()

for document in documents:
    document.metadata['filename'] = document.metadata['source']

Data-loss while decompressing corrupted data
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Data-loss while decompressing corrupted data
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from t

In [12]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/986 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [41]:
from pprint import pprint
pprint(testset)

TestDataset(test_data=[DataRow(question='What are the effects of the spell "Obscuring Mist"?', contexts=[' after completing the spell, if you wish. Treat this as a touch spell for which you are holding the charge (see page 176). You can hold the charge for as long as 1 round per level, at the end of which time the freezing sphere bursts centered on you (and you receive no saving throw to resist its effect). Firing the globe in a later round is a standard action.\n\nOrder’s Wrath Evocation [Lawful] Level: Law 4 Components: V, S Casting Time: 1 standard action Range: Medium (100 ft. + 10 ft./level) Area: Nonlawful creatures within a burst that fills a 30-ft. cube Duration: Instantaneous (1 round); see text Saving Throw: Will partial; see text Spell Resistance: Yes\n\nThis spell hides an object from location by divination (scrying) effects, such as the scrying spell or a crystal ball. Such an at- tempt automatically fails (if the divination is targeted on the object) or fails to per- ceiv

In [22]:
testset_pd = testset.to_pandas()

testset_pd

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"What are the effects of the spell ""Obscuring M...","[ after completing the spell, if you wish. Tre...",The spell creates a misty vapor that obscures ...,simple,[{'source': '../embed_docs/Player_s Handbook.p...,True
1,What is the purpose of an untrained Heal check...,"[, but if she misses by 5 or more, she falls (...",The purpose of an untrained Heal check is to m...,simple,[{'source': '../embed_docs/Player_s Handbook.p...,True
2,What are the consequences of delaying in terms...,[ : 159 \n\n159\n\n : 8 R E T P A ...,Your initiative result becomes the count on wh...,simple,[{'source': '../embed_docs/Player_s Handbook.p...,True
3,What are the advantages and disadvantages of b...,"[ Strength: Like dwarves, gnomes are tough, bu...","As a Small creature, a gnome gains a +1 size b...",simple,[{'source': '../embed_docs/Player_s Handbook.p...,True
4,What motivates sorcerers to go on adventures a...,[ either an unsubstantiated boast on the part ...,The typical sorcerer adventures in order to im...,simple,[{'source': '../embed_docs/Player_s Handbook.p...,True
5,What are the details of the Order's Wrath spell?,"[ after completing the spell, if you wish. Tre...",Order’s Wrath is an evocation spell of the Law...,reasoning,[{'source': '../embed_docs/Player_s Handbook.p...,True
6,How do conditions and modifiers impact skill c...,[ who is trained in them. If you don’t have Sp...,Conditions and modifiers can impact skill chec...,reasoning,[{'source': '../embed_docs/Player_s Handbook.p...,True
7,What are the effects of reading the text with ...,"[ tation is made, you cannot change it.\n\nA c...",nan,multi_context,[{'source': '../embed_docs/Player_s Handbook.p...,True
8,What does DC represent in skill checks and how...,[ who is trained in them. If you don’t have Sp...,DC represents the Difficulty Class in skill ch...,multi_context,[{'source': '../embed_docs/Player_s Handbook.p...,True
9,What is the purpose of flexible chainmail in a...,[ made of narrow vertical strips of metal rive...,Flexible chainmail in armor serves the purpose...,reasoning,[{'source': '../embed_docs/Player_s Handbook.p...,True


In [ ]:
import pandas as pd
from IPython.display import display, HTML
def df_pprint(df: pd.DataFrame):
  display(HTML(df.to_html().replace("\\n", "<br>")))
  
df_pprint(testset_pd)

In [42]:
from datasets import Dataset

testset_dataset = Dataset.from_pandas(testset_pd)

In [45]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate

result = evaluate(
    dataset=testset_dataset,
    metrics=[
        context_precision,
#        faithfulness,
#        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

The LLM did not return a valid classification.


{'context_precision': 0.9000, 'context_recall': 0.9841}

That was with generated data. Now to use ours.

In [96]:
from datasets import Dataset, Features, Sequence, Value

# Load the JSON data
import json
with open("../docs/groundtruths.json", "r") as f:
    data = json.load(f)

# Convert the list of dictionaries into a dictionary of lists
formatted_data = {
    "question": [entry["question"] for entry in data],
    "ground_truth": [entry["ground_truth"] for entry in data],
    "answer": [entry["answer"] for entry in data],
    "contexts": [json.loads(entry["contexts"].replace("'", "\"")) if isinstance(entry["contexts"], str) else entry["contexts"] for entry in data]
}

# Define the features
features = Features({
    "question": Value("string"),
    "ground_truth": Value("string"),
    "answer": Value("string"),
    "contexts": Sequence(Value("string"))
})

# Create the dataset
dataset = Dataset.from_dict(formatted_data, features=features)

# Check the dataset
print(dataset)

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 10
})


In [97]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate

result = evaluate(
    dataset=dataset,
    metrics=[context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 0.6735, 'faithfulness': 0.6400, 'answer_relevancy': 0.8659, 'context_recall': 0.6000}

In [101]:
from pprint import pprint
result_pd = result.to_pandas()

result_pd.to_excel("result.xlsx")